In [1]:
import requests
import bs4
from bs4 import BeautifulSoup
import pandas as pd
import time

In [2]:
# URL = 'https://www.indeed.com/jobs?q=data+scientist+%2420%2C000&l=New+York&start=10'
URL = 'https://www.indeed.co.uk/jobs?q=quantitative+analyst+%C2%A340,000&l=London'
#conducting a request of the stated URL above:
page = requests.get(URL)
#specifying a desired format of “page” using the html parser - this allows python to read the various components of the page, rather than treating it as one long string.
soup = BeautifulSoup(page.text, 'html.parser')
#printing soup in a more structured tree format that makes for easier reading
# print(soup.prettify())

In [3]:
def extract_salary(soup):
    salaries = []
    for div in soup.find_all(name = 'div', attrs = {'class':'row'}):
        try:
            salaries.append(div.find('nobr').text)
        except:
            try:
                div_two = div.find(name = 'div', attrs = {'class':'sjcl'})
                div_three = div_two.find('div')
                salaries.append(div_three.text.strip())
#                 print(div_three)
            except:
                salaries.append('Nothing_found')    
    return(salaries)

def extract_summary(soup):
    summaries = []
    divs = soup.find_all('div', attrs = {'class':'summary'})
#     print(divs)
    for i, div in enumerate(divs):
#         print(i, div.text)
        summaries.append(div.text.strip())
    return(summaries)

def extract_location(soup):
    locations = []
    spans = soup.find_all('span', attrs={'class':'location'})
    for span in spans:
        locations.append(span.text)
    return(locations)

def extract_company(soup):
    companies = []
    for div in soup.find_all(name = 'div', attrs = {'class':'row'}):
        company = div.find_all(name = 'span', attrs = {'class':'company'})
        if len(company) > 0:
            for b in company:
                companies.append(b.text.strip())
            else:
                sec_try = div.find_all(name = 'span', attrs = {'class':'result-link-source'})
                for span in sec_try:
                    companies.append(span.text.strip())
    return(companies)

def extract_job_title(soup):
    jobs = []
    for div in soup.find_all(name='div', attrs = {'class':'row'}):
        for a in div.find_all(name = 'a', attrs = {'data-tn-element':"jobTitle"}):
            jobs.append(a['title'])
    return(jobs)

URL = 'https://www.indeed.co.uk/jobs?q=quantitative+analyst+%C2%A340,000&l=London'
page = requests.get(URL)
soup = BeautifulSoup(page.text, 'html.parser')

title = extract_job_title(soup)

company = extract_company(soup)

ls = extract_location(soup)

summary = extract_summary(soup)
    
salary = extract_salary(soup)

df1 = zip(company, title, ls, salary, summary)
df = pd.DataFrame(df1, columns = ['company', "title", 'location', 'salary', 'summary'])
df.head()

,company,title,location,salary,summary
0,Spencer Clarke Group,Behavioral Insight Analyst,London,Spencer Clarke Group\n\n\n\n4.7,Use quantitative and qualitative skills to lea...
1,Selby Jennings,Model Validation - Quantitative Analyst,London,Selby Jennings,Help build out and maintain the internal quant...
2,Selby Jennings,FO Quantitative Analyst - Pricing,London SW1E 5LB,Selby Jennings,Excellent understanding of Stochastic Calculus...
3,"Nomura Holdings, Inc.",Quantitative Strategies Analyst,London,"Nomura Holdings, Inc.\n\n\n\n3.9",Higher degree in quantitative discipline such ...
4,Cambridge Associates LLC,Investment Analyst,London,Cambridge Associates LLC,Although the position involves quantitative as...


In [4]:
## fetching metadata
max_jobs_to_fetch = 100
columns = ['location', 'company_name', 'job_title', 'summary']
# columns = ['company', 'title']
df = pd.DataFrame(columns = columns)

for start in range(0, max_jobs_to_fetch, 10):
    try:
        url = 'https://www.indeed.co.uk/jobs?q=finance&l=London&radius=25&start='
        page = requests.get(url + str(start))
        print('retriving url: ', (url + str(start)))
    except:
        print('---Failed to retrieve---')
        print('url:', (url + str(start)))
        continue
    soup = BeautifulSoup(page.text, 'html.parser')
    # extract info from class:row
    # company name and job title
    companies, jobs = [], []
    for div in soup.find_all(name = 'div', attrs = {'class':'row'}):
        company = div.find_all(name = 'span', attrs = {'class':'company'})
        if len(company) > 0:
            for b in company:
                companies.append(b.text.strip())
            else:
                sec_try = div.find_all(name = 'span', attrs = {'class':'result-link-source'})
                for span in sec_try:
                    companies.append(span.text.strip())
        for a in div.find_all(name = 'a', attrs = {'data-tn-element':"jobTitle"}):
            jobs.append(a['title'])
            
    # extract location
    locations = []
    spans = soup.find_all('span', attrs={'class':'location'})
    for span in spans:
        locations.append(span.text)
        
    # extract summaries
    summaries = []
    divs = soup.find_all('div', attrs = {'class':'summary'})
    for i, div in enumerate(divs):
        summaries.append(div.text.strip())
        
    df_temp = list(zip(locations, companies, jobs, summaries))
    df_temp = pd.DataFrame(df_temp, columns = columns)
    df = df.append(df_temp).reset_index(drop = True)
    
# df.to_csv('sample_data.csv', encoding='utf-8')
df.head()

retriving url:  https://www.indeed.co.uk/jobs?q=finance&l=London&radius=25&start=0
retriving url:  https://www.indeed.co.uk/jobs?q=finance&l=London&radius=25&start=10
retriving url:  https://www.indeed.co.uk/jobs?q=finance&l=London&radius=25&start=20
retriving url:  https://www.indeed.co.uk/jobs?q=finance&l=London&radius=25&start=30
retriving url:  https://www.indeed.co.uk/jobs?q=finance&l=London&radius=25&start=40
retriving url:  https://www.indeed.co.uk/jobs?q=finance&l=London&radius=25&start=50
retriving url:  https://www.indeed.co.uk/jobs?q=finance&l=London&radius=25&start=60
retriving url:  https://www.indeed.co.uk/jobs?q=finance&l=London&radius=25&start=70
retriving url:  https://www.indeed.co.uk/jobs?q=finance&l=London&radius=25&start=80
retriving url:  https://www.indeed.co.uk/jobs?q=finance&l=London&radius=25&start=90


,location,company_name,job_title,summary
0,London SW1P,TAB Asset Management,WORKING FROM HOME - Trainee Financial Broker,Long term career in wealth management.\nFundin...
1,London,Cyber Tech Company,Intelligence Analyst,Schedule: * Full-time (09.00 – 18.00).\nSilobr...
2,London,Barnsdales Fm Ltd,Facilities Management/Finance Admin Coordinator,To provide site team leadership and liaise wit...
3,South East London,Hamberley Care,Accounts Payable,Other ad hoc finance and office administrative...
4,Mayfair,Harris Federation,Finance Administrative Assistant - Part Time,We are currently looking to appoint a Finance ...


In [5]:
max_jobs_to_fetch = 300
columns = ['location', 'company_name', 'job_title', 'summary', 'full_info', 'ref']
# columns = ['company', 'title']
df = pd.DataFrame(columns = columns)

url_list = {1:'https://www.indeed.co.uk/jobs?q=finance&l=London&radius=25&start=', 
            2:'https://www.indeed.co.uk/jobs?q=telecommunications&l=London&radius=100&start='}

for index in url_list:
    print(index, url_list[index])
    for start in range(0, max_jobs_to_fetch, 10):
        try:
            url = url_list[index] + str(start)
            page = requests.get(url)
            print('retriving url: ', url)
        except:
            print('---Failed to retrieve---')
            print('url:', url)
            continue
        soup = BeautifulSoup(page.text, 'html.parser')
        time.sleep(1)
        ## metadata from mainpage

        # extract info from class:row
        # company name and job title
        companies, jobs = [], []
        for div in soup.find_all(name = 'div', attrs = {'class':'row'}):
            company = div.find_all(name = 'span', attrs = {'class':'company'})
            if len(company) > 0:
                for b in company:
                    companies.append(b.text.strip())
                else:
                    sec_try = div.find_all(name = 'span', attrs = {'class':'result-link-source'})
                    for span in sec_try:
                        companies.append(span.text.strip())
            for a in div.find_all(name = 'a', attrs = {'data-tn-element':"jobTitle"}):
                jobs.append(a['title'])

        # extract location
        locations = []
        spans = soup.find_all('span', attrs={'class':'location'})
        for span in spans:
            locations.append(span.text)

        # extract summaries
        summaries = []
        divs = soup.find_all('div', attrs = {'class':'summary'})
        for i, div in enumerate(divs):
            summaries.append(div.text.strip())

        ## crawl to subpages
        descriptions = []
        link_list = []
        for adlink in soup.select('a[onmousedown*="return rclk(this,jobmap["]'):
            suburl = "https://www.indeed.com" + adlink['href']
            link_list.append(suburl)
            try:
                subpage = requests.get(suburl)
                subsoup = BeautifulSoup(subpage.text)
            except:
                print('--- Failed to retrieve sub-URL ---')
                print('url: ', suburl)
            # extract descriptions
            for des in subsoup.select('div[class*="jobsearch-JobComponent-description"]'): 
                 descriptions.append(des.get_text())

        df_temp = zip(locations, companies, jobs, summaries, descriptions, link_list)
        df_temp = pd.DataFrame(df_temp, columns = columns)
        df = df.append(df_temp).reset_index(drop = True)

1 https://www.indeed.co.uk/jobs?q=finance&l=London&radius=25&start=
retriving url:  https://www.indeed.co.uk/jobs?q=finance&l=London&radius=25&start=0
retriving url:  https://www.indeed.co.uk/jobs?q=finance&l=London&radius=25&start=10
retriving url:  https://www.indeed.co.uk/jobs?q=finance&l=London&radius=25&start=20
retriving url:  https://www.indeed.co.uk/jobs?q=finance&l=London&radius=25&start=30
retriving url:  https://www.indeed.co.uk/jobs?q=finance&l=London&radius=25&start=40
retriving url:  https://www.indeed.co.uk/jobs?q=finance&l=London&radius=25&start=50
retriving url:  https://www.indeed.co.uk/jobs?q=finance&l=London&radius=25&start=60
retriving url:  https://www.indeed.co.uk/jobs?q=finance&l=London&radius=25&start=70
retriving url:  https://www.indeed.co.uk/jobs?q=finance&l=London&radius=25&start=80
retriving url:  https://www.indeed.co.uk/jobs?q=finance&l=London&radius=25&start=90
retriving url:  https://www.indeed.co.uk/jobs?q=finance&l=London&radius=25&start=100
retrivin

In [9]:
df

,location,company_name,job_title,summary,full_info,ref
0,Walworth,TAB Asset Management,WORKING FROM HOME - Trainee Financial Broker,Long term career in wealth management.\nFundin...,We have an exciting opportunity to join our Fi...,https://www.indeed.com/rc/clk?jk=6b73c632767df...
1,London,Walterton & Elgin Community Homes,Accounts Assistant,Experience in a finance or accounting role (de...,You're the brains behind our work.\nYou're rea...,https://www.indeed.com/rc/clk?jk=ff1d0f140be90...
2,Mayfair,Harris Federation,Finance Administrative Assistant - Part Time,We are currently looking to appoint a Finance ...,Main functions of the job: The reconciliation ...,https://www.indeed.com/company/United-Bank-UK/...
3,London EC1V,HANetf,London Based Finance assetmanagement fund Sale...,We are looking for several interns to help sup...,We are looking for a Junior Finance Analyst wi...,https://www.indeed.com/rc/clk?jk=c9c4725473750...
4,London,Abellio London,Finance Assistant,The successful applicant will report to the Fi...,Company\nBrookfield is a global alternative as...,https://www.indeed.com/rc/clk?jk=0dbd9eb04d38e...
...,...,...,...,...,...,...
693,Gillingham,experisuk,Fibre Surveyor - London x 2,"FTTP, ISP, Fibre Optic, Broadband Networks & T...",HELP BRITAIN STAY CONNECTED\nIt’s a strange ti...,https://www.indeed.com/rc/clk?jk=b3ea7eb36abbe...
694,London,PeopleFirst Recruitment,Japanese job-IT Pre-sales,Manage and assist with Pre-sales - Solutions P...,Chinese Job-Sales Support Operations & Service...,https://www.indeed.com/rc/clk?jk=4d09715a39209...
695,London,PeopleFirst Recruitment,"Sales Manager (Enterprise, Telecommunication)",Knowledge on telecommunication industry in Asi...,8+ years of experience in Architecture and Des...,https://www.indeed.com/rc/clk?jk=336ed9d2ca240...
696,London,CityFibre,Fibre Supervisor,We’re looking for a Fibre Supervisor to monito...,Chinese job-Telecoms Customer Solutions-rj\nCo...,https://www.indeed.com/rc/clk?jk=344c341c6cd7c...


In [11]:
df_600 = df.iloc[0:300, :].copy()
df_600 = pd.concat([df_600, df.iloc[300:600, :].copy()])
df_600.shape

(600, 6)

In [13]:
df.to_csv('../data/indeed_finance_telecom.csv', encoding = 'utf-8')
df_600.to_csv('../data/indeed_finance_telecom_600.csv', encoding = 'utf-8')